<a href="https://colab.research.google.com/github/hugopoggi/enem/blob/main/impacto_pandemia_enem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=f8765ff7ef7943f4b6af79c84d54972ff9235ec702a427ba2e1125f0bfe5d2f7
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession as ss
from pyspark.sql import functions as f
from pyspark.sql.types import DoubleType

In [3]:
spark = ss.builder.appName("ENEM").getOrCreate()

**INICIANDO TRATAMENTO DE DADOS ENEM 2019
**

In [4]:
# Importando a base de dados do enem de 2019
url = "/content/drive/MyDrive/BasesSpark/enem/enem_2019/DADOS/MICRODADOS*"
enem_2019 =spark.read.csv(
    url,
    header=True,
    inferSchema=True,
    encoding= "UTF-8",
    sep=";"
)

In [5]:
enem_2019.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+----------------+---------+---------+----------------------+------------------+---------------+------------------+------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_AN

In [6]:
enem_2019.count()

5095171

In [7]:
enem_2019.printSchema()

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- TP_FAIXA_ETARIA: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: integer 

In [8]:
# Filtrando alunos que estiveram presentes em todos os dias de prova, que nao concluiram ou irão concluir o ensino medio
enem_2019.where(
    "TP_FAIXA_ETARIA <= 10 and TP_PRESENCA_CN like '1' and TP_PRESENCA_CH like '1' and TP_PRESENCA_LC like '1' and TP_PRESENCA_MT like '1' and TP_ST_CONCLUSAO like '2' or TP_ST_CONCLUSAO like '3'"
).show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+--------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO

In [9]:
# Aplicando alteração ao Data Frame
enem_2019 = enem_2019.where(
    "TP_FAIXA_ETARIA <= 10 and TP_PRESENCA_CN like '1' and TP_PRESENCA_CH like '1' and TP_PRESENCA_LC like '1' and TP_PRESENCA_MT like '1' and TP_ST_CONCLUSAO like '2' or TP_ST_CONCLUSAO like '3'"
)

In [10]:
enem_2019.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+--------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO

In [13]:
# Removendo notas nulas:
enem_2019.dropna(how='any',subset=["NU_NOTA_CN",
                                   "NU_NOTA_CH",
                                   "NU_NOTA_LC",
                                   "NU_NOTA_MT",
                                   "NU_NOTA_REDACAO"]).show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+--------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO

In [14]:
# Fazendo backup
bkp = enem_2019

In [15]:
# Aplicando alteração:
enem_2019 = enem_2019.dropna(how='any',subset=["NU_NOTA_CN",
                                   "NU_NOTA_CH",
                                   "NU_NOTA_LC",
                                   "NU_NOTA_MT",
                                   "NU_NOTA_REDACAO"])

In [18]:
# Validando a alteração: verificando se existem valores nulos nas colunas especificadas
enem_2019.filter((enem_2019["NU_NOTA_CN"].isNull()) |
                 (enem_2019["NU_NOTA_CH"].isNull()) |
                 (enem_2019["NU_NOTA_LC"].isNull()) |
                 (enem_2019["NU_NOTA_MT"].isNull()) |
                 (enem_2019["NU_NOTA_REDACAO"].isNull())).show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+----------------+---------+---------+----------------------+------------------+---------------+------------------+------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+---------------+---------------+---------------+---------------+---------+--------------+--------------+--------------+--------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|TP_ENSINO|IN_TREINEIRO|

In [19]:
# Filtrando colunas que não interessam para esta analise:
enem_2019.drop("TP_ANO_CONCLUIU",
               "TP_ST_CONCLUSAO",
               "CO_MUNICIPIO_ESC",
               "CO_UF_ESC",
               "CO_MUNICIPIO_PROVA",
               "NO_MUNICIPIO_ESC",
               "TP_SIT_FUNC_ESC",
               "TP_ENSINO",
               "TX_RESPOSTAS_CN",
               "TX_RESPOSTAS_CH",
               "TX_RESPOSTAS_LC",
               "TX_RESPOSTAS_MT",
               "TP_LINGUA",
               "TX_GABARITO_CN",
               "TX_GABARITO_CH",
               "TX_GABARITO_LC",
               "TX_GABARITO_MT",
               "TP_STATUS_REDACAO",
               "NU_NOTA_COMP1",
               "NU_NOTA_COMP2",
               "NU_NOTA_COMP3",
               "NU_NOTA_COMP4",
               "NU_NOTA_COMP5",
               "CO_UF_PROVA",
               "SG_UF_PROVA",
               "TP_PRESENCA_CN",
               "TP_PRESENCA_CH",
               "TP_PRESENCA_LC",
               "TP_PRESENCA_MT",
               "CO_PROVA_CN",
               "CO_PROVA_CH",
               "CO_PROVA_LC",
               "CO_PROVA_MT",
               "Q001",
               "Q002",
               "Q003",
               "Q004",
               "Q005",
               "Q006",
               "Q007",
               "Q008",
               "Q009",
               "Q010",
               "Q011",
               "Q012",
               "Q013",
               "Q014",
               "Q015",
               "Q016",
               "Q017",
               "Q018",
               "Q019",
               "Q020",
               "Q021",
               "Q022",
               "Q023",
               "Q024",
               "Q025").show()





+------------+------+---------------+-------+---------------+-----------+----------------+---------+------------+---------+----------------------+------------------+--------------------+----------+----------+----------+----------+---------------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ESCOLA|IN_TREINEIRO|SG_UF_ESC|TP_DEPENDENCIA_ADM_ESC|TP_LOCALIZACAO_ESC|  NO_MUNICIPIO_PROVA|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|NU_NOTA_REDACAO|
+------------+------+---------------+-------+---------------+-----------+----------------+---------+------------+---------+----------------------+------------------+--------------------+----------+----------+----------+----------+---------------+
|190001592266|  2019|              2|      M|              1|          1|               1|        2|           0|     NULL|                  NULL|              NULL|           S�o Paulo|     543.9|     548.1|     502.5|     480.7|            400|
|19000142154

In [20]:
# Aplicando Data frame
enem_2019 = enem_2019.drop(
               "TP_ANO_CONCLUIU",
               "TP_ESTADO_CIVIL",
               "TP_ST_CONCLUSAO",
               "CO_MUNICIPIO_ESC",
               "CO_UF_ESC",
               "CO_MUNICIPIO_PROVA",
               "NO_MUNICIPIO_ESC",
               "TP_SIT_FUNC_ESC",
               "TP_ENSINO",
               "TX_RESPOSTAS_CN",
               "TX_RESPOSTAS_CH",
               "TX_RESPOSTAS_LC",
               "TX_RESPOSTAS_MT",
               "TP_LINGUA",
               "TX_GABARITO_CN",
               "TX_GABARITO_CH",
               "TX_GABARITO_LC",
               "TX_GABARITO_MT",
               "TP_STATUS_REDACAO",
               "NU_NOTA_COMP1",
               "NU_NOTA_COMP2",
               "NU_NOTA_COMP3",
               "NU_NOTA_COMP4",
               "NU_NOTA_COMP5",
               "CO_UF_PROVA",
               "SG_UF_PROVA",
               "TP_PRESENCA_CN",
               "TP_PRESENCA_CH",
               "TP_PRESENCA_LC",
               "TP_PRESENCA_MT",
               "CO_PROVA_CN",
               "CO_PROVA_CH",
               "CO_PROVA_LC",
               "CO_PROVA_MT",
               "Q001",
               "Q002",
               "Q003",
               "Q004",
               "Q005",
               "Q006",
               "Q007",
               "Q008",
               "Q009",
               "Q010",
               "Q011",
               "Q012",
               "Q013",
               "Q014",
               "Q015",
               "Q016",
               "Q017",
               "Q018",
               "Q019",
               "Q020",
               "Q021",
               "Q022",
               "Q023",
               "Q024",
               "Q025")

In [21]:
enem_2019.show()

+------------+------+---------------+-------+-----------+----------------+---------+------------+---------+----------------------+------------------+--------------------+----------+----------+----------+----------+---------------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_COR_RACA|TP_NACIONALIDADE|TP_ESCOLA|IN_TREINEIRO|SG_UF_ESC|TP_DEPENDENCIA_ADM_ESC|TP_LOCALIZACAO_ESC|  NO_MUNICIPIO_PROVA|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|NU_NOTA_REDACAO|
+------------+------+---------------+-------+-----------+----------------+---------+------------+---------+----------------------+------------------+--------------------+----------+----------+----------+----------+---------------+
|190001592266|  2019|              2|      M|          1|               1|        2|           0|     NULL|                  NULL|              NULL|           S�o Paulo|     543.9|     548.1|     502.5|     480.7|            400|
|190001421547|  2019|              4|      M|          3|               1|  

In [22]:
# Renomenando colunas:
enem_2019.withColumnsRenamed({
    "NU_INSCRICAO":"inscricao",
    "NU_ANO":"ano_prova",
    "TP_FAIXA_ETARIA":"faixa_etaria",
    "TP_SEXO":"genero",
    "TP_COR_RACA":"cor_raca",
    "TP_NACIONALIDADE":"nacionalidade",
    "TP_ESCOLA": "tipo_escola",
    "IN_TREINEIRO": "treineiro",
    "SG_UF_ESC":"uf_escola",
    "TP_DEPENDENCIA_ADM_ESC":"dependencia_esc",
    "TP_LOCALIZACAO_ESC":"localizacao_esc",
    "NO_MUNICIPIO_PROVA":"municipio_esc",
    "NU_NOTA_CN":"n_ciencias_nat",
    "NU_NOTA_CH":"n_ciencias_hum",
    "NU_NOTA_LC":"n_linguagens_cod",
    "NU_NOTA_MT":"n_matematica",
    "NU_NOTA_REDACAO":"n_redacao"
}).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|           2|     M|       1|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|           4|     M|       3|            1|          1|        1|     NULL|         

In [23]:
# Aplicando alterações
enem_2019=enem_2019.withColumnsRenamed({
    "NU_INSCRICAO":"inscricao",
    "NU_ANO":"ano_prova",
    "TP_FAIXA_ETARIA":"faixa_etaria",
    "TP_SEXO":"genero",
    "TP_COR_RACA":"cor_raca",
    "TP_NACIONALIDADE":"nacionalidade",
    "TP_ESCOLA": "tipo_escola",
    "IN_TREINEIRO": "treineiro",
    "SG_UF_ESC":"uf_escola",
    "TP_DEPENDENCIA_ADM_ESC":"dependencia_esc",
    "TP_LOCALIZACAO_ESC":"localizacao_esc",
    "NO_MUNICIPIO_PROVA":"municipio_esc",
    "NU_NOTA_CN":"n_ciencias_nat",
    "NU_NOTA_CH":"n_ciencias_hum",
    "NU_NOTA_LC":"n_linguagens_cod",
    "NU_NOTA_MT":"n_matematica",
    "NU_NOTA_REDACAO":"n_redacao"
})

In [24]:
enem_2019.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|           2|     M|       1|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|           4|     M|       3|            1|          1|        1|     NULL|         

In [27]:
# Alterando valores das colunas segundo dicionario de dados
enem_2019.withColumn(    "faixa_etaria",
                         f.when(enem_2019.faixa_etaria == 1, "MENOR QUE 17")\
                         .when(enem_2019.faixa_etaria == 2, "17")\
                         .when(enem_2019.faixa_etaria == 3, "18")\
                         .when(enem_2019.faixa_etaria == 4, "19")\
                         .when(enem_2019.faixa_etaria == 5, "20")\
                         .when(enem_2019.faixa_etaria == 6, "21")\
                         .when(enem_2019.faixa_etaria == 7, "22")\
                         .when(enem_2019.faixa_etaria == 8, "23")\
                         .when(enem_2019.faixa_etaria == 9, "24")\
                         .when(enem_2019.faixa_etaria == 10, "25").otherwise("N/I")
                         ).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|         N/I|     M|       1|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|         N/I|     M|       3|            1|          1|        1|     NULL|         

In [28]:
# Implementando alteração:
enem_2019 = enem_2019.withColumn(    "faixa_etaria",
                         f.when(enem_2019.faixa_etaria == 1, "MENOR QUE 17")\
                         .when(enem_2019.faixa_etaria == 2, "17")\
                         .when(enem_2019.faixa_etaria == 3, "18")\
                         .when(enem_2019.faixa_etaria == 4, "19")\
                         .when(enem_2019.faixa_etaria == 5, "20")\
                         .when(enem_2019.faixa_etaria == 6, "21")\
                         .when(enem_2019.faixa_etaria == 7, "22")\
                         .when(enem_2019.faixa_etaria == 8, "23")\
                         .when(enem_2019.faixa_etaria == 9, "24")\
                         .when(enem_2019.faixa_etaria == 10, "25").otherwise("N/I")
                         )

In [29]:
enem_2019.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|         N/I|     M|       1|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|         N/I|     M|       3|            1|          1|        1|     NULL|         

In [30]:
# Alterando valores das colunas segundo dicionario de dados
enem_2019.withColumn("cor_raca",
                         f.when(enem_2019.cor_raca == 1, "BRANCA")\
                         .when(enem_2019.cor_raca == 2, "PRETA")\
                         .when(enem_2019.cor_raca == 3, "PARDA")\
                         .when(enem_2019.cor_raca == 4, "AMARELA")
                         .when(enem_2019.cor_raca == 5, "INDIGENA").otherwise("N/I")
                         ).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|         N/I|     M|  BRANCA|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|         N/I|     M|   PARDA|            1|          1|        1|     NULL|         

In [31]:
# Implementando alteração
enem_2019 = enem_2019.withColumn("cor_raca",
                         f.when(enem_2019.cor_raca == 1, "BRANCA")\
                         .when(enem_2019.cor_raca == 2, "PRETA")\
                         .when(enem_2019.cor_raca == 3, "PARDA")\
                         .when(enem_2019.cor_raca == 4, "AMARELA")
                         .when(enem_2019.cor_raca == 5, "INDIGENA").otherwise("N/I")
                         )

In [32]:
enem_2019.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|         N/I|     M|  BRANCA|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|         N/I|     M|   PARDA|            1|          1|        1|     NULL|         

In [33]:
# Implementando alteração
enem_2019.withColumn("nacionalidade",
                         f.when(enem_2019.nacionalidade == 1, "BRASILEIRO(A)")\
                         .when(enem_2019.nacionalidade == 2, "BRASILEIRO(A) NATURALIZADO")\
                         .when(enem_2019.nacionalidade == 3, "ESTRANGEIRO")\
                         .when(enem_2019.nacionalidade == 4, "BRASILEIRO, NASCIDO NO EXTERIOR").otherwise("N/I")
                         ).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|         N/I|     M|  BRANCA|       BRASILEIRO(A)|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|         N/I|     M|   PARDA|       BRASILEIRO(A)|      

In [34]:
# Implementando normalização:
enem_2019 = enem_2019.withColumn("nacionalidade",
                         f.when(enem_2019.nacionalidade == 1, "BRASILEIRO(A)")\
                         .when(enem_2019.nacionalidade == 2, "BRASILEIRO(A) NATURALIZADO")\
                         .when(enem_2019.nacionalidade == 3, "ESTRANGEIRO")\
                         .when(enem_2019.nacionalidade == 4, "BRASILEIRO, NASCIDO NO EXTERIOR").otherwise("N/I")
                         )

In [35]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|         N/I|     M|  BRANCA|       BRASILEIRO(A)|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|         N/I|     M|   PARDA|       BRASILEIRO(A)|      

In [36]:
# Alterando valores das colunas segundo dicionario:
enem_2019.withColumn("tipo_escola",
                         f.when(enem_2019.tipo_escola == 2, "PUBLICA")\
                         .when(enem_2019.tipo_escola == 3, "PRIVADA")\
                         .when(enem_2019.tipo_escola == 4, "EXTERIOR")
).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|         N/I|     M|  BRANCA|       BRASILEIRO(A)|    PUBLICA|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|         N/I|     M|   PARDA|       BRASILEIRO(A)|      

In [37]:
enem_2019 = enem_2019.withColumn("tipo_escola",
                         f.when(enem_2019.tipo_escola == 2, "PUBLICA")\
                         .when(enem_2019.tipo_escola == 3, "PRIVADA")\
                         .when(enem_2019.tipo_escola == 4, "EXTERIOR")
)

In [38]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|         N/I|     M|  BRANCA|       BRASILEIRO(A)|    PUBLICA|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|         N/I|     M|   PARDA|       BRASILEIRO(A)|      

In [39]:
# Removendo valores nulos do tipo escola:
enem_2019.dropna(subset=["tipo_escola"]).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|         N/I|     M|  BRANCA|       BRASILEIRO(A)|    PUBLICA|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001867757|     2019|         N/I|     F|  BRANCA|       BRASILEIRO(A)|    PR

In [40]:
# Aplicando alteração:
enem_2019 = enem_2019.dropna(subset=["tipo_escola"])

In [42]:
# Verificando alteração:
enem_2019.filter((enem_2019["tipo_escola"].isNull())).show()

+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
|inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+



In [43]:
# Alterando dados segundo dicionario
enem_2019.withColumn("treineiro",
                         f.when(enem_2019.treineiro == 1, "SIM").otherwise("NAO")
                         ).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|         N/I|     M|  BRANCA|       BRASILEIRO(A)|    PUBLICA|      NAO|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001867757|     2019|         N/I|     F|  BRANCA|       BRASILEIRO(A)|    PR

In [45]:
enem_2019=enem_2019.withColumn("treineiro",
                         f.when(enem_2019.treineiro == 1, "SIM").otherwise("NAO"))

In [46]:
# Atualizando backup
bpk = enem_2019

In [47]:
# Tratando dados "uf_escola" e "municipio_esc" - se "uf_escola" e "municipio_esc"
# for nulo remova a linha
enem_2019.dropna(thresh=2,subset=["uf_escola","dependencia_esc"]).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|         N/I|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|              4|              1|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|         N/I|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [48]:
enem_2019 = enem_2019.dropna(thresh=2,subset=["uf_escola","dependencia_esc"])

In [49]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|         N/I|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|              4|              1|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|         N/I|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [50]:
# Alterando dados de colunas segundo dicionario:
enem_2019.withColumn("dependencia_esc",
                         f.when(enem_2019.dependencia_esc == 1, "FEDERAL")\
                         .when(enem_2019.dependencia_esc == 2, "MUNICIPAL")\
                         .when(enem_2019.dependencia_esc == 3, "PRIVADA").otherwise("N/I")
).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|         N/I|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|              1|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|         N/I|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [51]:
enem_2019 = enem_2019.withColumn("dependencia_esc",
                         f.when(enem_2019.dependencia_esc == 1, "FEDERAL")\
                         .when(enem_2019.dependencia_esc == 2, "MUNICIPAL")\
                         .when(enem_2019.dependencia_esc == 3, "PRIVADA").otherwise("N/I")
)

In [52]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|         N/I|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|              1|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|         N/I|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [53]:
# Alterando dados de colunas segundo dicionario:
enem_2019.withColumn("localizacao_esc",
                         f.when(enem_2019.localizacao_esc == 1, "URBANO")\
                         .when(enem_2019.localizacao_esc == 2, "RURAL")
).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|         N/I|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|         URBANO|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|         N/I|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [54]:
enem_2019 = enem_2019.withColumn("localizacao_esc",
                         f.when(enem_2019.localizacao_esc == 1, "URBANO")\
                         .when(enem_2019.localizacao_esc == 2, "RURAL")
)

In [55]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|         N/I|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|         URBANO|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|         N/I|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [56]:
bkp= enem_2019

In [58]:
# Removendo dados nulos da coluna "localizacao_esc":
enem_2019 = enem_2019.dropna(subset=['localizacao_esc'])

In [60]:
# Verificando alteração
enem_2019.filter((enem_2019['localizacao_esc'].isNull())).show()

+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
|inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+



In [63]:
bkp = enem_2019

In [66]:
# Adicionando a coluna media final para enem:
enem_2019 = enem_2019.withColumn("nota_final",
                                  (f.col("n_ciencias_nat") +
                                   f.col("n_ciencias_hum") +
                                   f.col("n_linguagens_cod") +
                                   f.col("n_matematica") +
                                   f.col("n_redacao")) / 5)

In [67]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+------------------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|        nota_final|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+------------------+
|190001867757|     2019|         N/I|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|         URBANO|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|            722.58|
|190

In [68]:
bkp = enem_2019

In [69]:
# normalizando coluna nota final deixando 2 casa apos o . :
enem_2019 = enem_2019.withColumn("nota_final",
                                 f.format_number("nota_final", 2))

In [70]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+----------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|nota_final|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+----------+
|190001867757|     2019|         N/I|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|         URBANO|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|    722.58|
|190001692704|     2019|         N/I

In [71]:
enem_2019.count()

940926

In [72]:
# Criando view
enem_2019.createOrReplaceTempView("v_enem_2019")

In [ ]:
spark.sql(
"""
SELECT A.uf_escola, AVG(A.nota_final) AS media_estado
FROM v_enem_2019 A
GROUP BY uf_escola
ORDER BY media_estado DESC
"""
).show()